# 01 - Data Exploration
# AutonomousVehiclePerception/notebooks/01_data_exploration.ipynb

Explore KITTI and nuScenes datasets for autonomous vehicle perception.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.data.kitti_dataset import (
    KITTIDataset, KITTI_CLASSES, CLASS_TO_IDX,
    voxelize_points, load_lidar_points
)
from src.data.nuscenes_dataset import NuScenesDataset, NUSCENES_CLASSES, CAMERA_NAMES

print('KITTI Classes:', KITTI_CLASSES)
print('nuScenes Classes:', NUSCENES_CLASSES)

## KITTI Dataset Structure

Set `KITTI_ROOT` to your local KITTI download path.

In [ ]:
KITTI_ROOT = Path('../data/raw/kitti')  # Update this path

if KITTI_ROOT.exists():
    dataset = KITTIDataset(root=KITTI_ROOT, split='training')
    print(f'KITTI samples: {len(dataset)}')
    
    # Load first sample
    image, target = dataset[0]
    print(f'Image shape: {image.shape}')
    print(f'Annotations: {target["num_objects"]} objects')
    for ann in target['annotations'][:5]:
        print(f"  {ann['class']}: bbox={ann['bbox_2d']}")
else:
    print(f'KITTI not found at {KITTI_ROOT}')
    print('Download from: http://www.cvlibs.net/datasets/kitti/eval_object.php')

## LiDAR Voxelization Demo

In [ ]:
# Demo voxelization with synthetic point cloud
np.random.seed(42)
fake_points = np.random.randn(50000, 4).astype(np.float32)
fake_points[:, :3] *= 20

voxel_grid = voxelize_points(fake_points)
print(f'Voxel grid shape: {voxel_grid.shape}')
print(f'Occupied voxels: {(voxel_grid > 0).sum()} / {voxel_grid.size}')

# Visualize BEV (bird's eye view)
bev = voxel_grid.max(axis=0)  # collapse depth
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(bev, cmap='hot', origin='lower')
ax.set_title('Bird\'s Eye View - Voxel Occupancy')
ax.set_xlabel('X bins')
ax.set_ylabel('Y bins')
plt.colorbar(ax.images[0], label='Point count')
plt.tight_layout()
plt.show()

## Augmentation Pipeline Preview

In [ ]:
from src.data.augmentations import get_train_transforms, get_val_transforms

train_tf = get_train_transforms(image_size=(480, 640))
val_tf = get_val_transforms(image_size=(480, 640))

# Create synthetic image
dummy_img = np.random.randint(0, 255, (720, 1280, 3), dtype=np.uint8)
dummy_bboxes = [[200, 300, 500, 600]]
dummy_labels = [0]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
for i, ax in enumerate(axes.flat):
    result = train_tf(image=dummy_img, bboxes=dummy_bboxes, class_labels=dummy_labels)
    # Denormalize for display
    img = result['image'].permute(1, 2, 0).numpy()
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = np.clip(img * std + mean, 0, 1)
    ax.imshow(img)
    ax.set_title(f'Augmentation {i+1}')
    ax.axis('off')
plt.suptitle('Training Augmentation Samples')
plt.tight_layout()
plt.show()